<a href="https://colab.research.google.com/github/HayyanWasi/openai-agent-learning/blob/main/07_handoffs_agentsdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai google-generativeai openai-agents python-dotenv


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from agents import Agent, Runner, AsyncOpenAI, set_default_openai_client, set_tracing_disabled, OpenAIChatCompletionsModel, RunConfig
import os


from google.colab import userdata


gemini_api_key = userdata.get('GEMINI_API_KEY')
set_tracing_disabled(True)

external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/",
)


set_default_openai_client(external_client)


model = OpenAIChatCompletionsModel(
    model="models/gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
    )


In [ ]:
from pydantic import BaseModel

class Tutorial(BaseModel):
  outline:str
  tutorial:str



#model which is used by the agent to generate tutorial
tutorial_genrator = Agent(
    name="Tutorial Generator",
    # handsoff_description = "this is used for making tutorials based on outline builder", #use for precision in decision making
    instructions="your are You are TutorialGPT, an expert in generating clear, structured, and engaging tutorials on any topic. Your job is to create high-quality step-by-step tutorials tailored to the target audience’s skill level (beginner, intermediate, or advanced).",
    model=model,
    output_type=Tutorial
)
#used to generate outline after completing its work it can handoff its control to tutorial_generator to make tutorial around the outline after giving handoff an agent can not be return bacl
outline_builder= Agent(
    name="outline border",
    instructions=("You are OutlineGPT, an expert assistant that creates well-structured, logical outlines for programming tutorials."
    "Your task is to generate detailed tutorial outlines on any programming topic (language, framework, tool, or concept) based on the user’s input."),
    handoffs=[tutorial_genrator], #using handoff
    model=model
)
tutorial_response = await Runner.run(outline_builder,"i want to learn  loops in java")
print(tutorial_response.final_output)

In [ ]:
from pydantic import BaseModel
from agents import function_tool
from agents import RunContextWrapper, handoff
from google.generativeai import configure, GenerativeModel

class ManageEscalation(BaseModel):
    issue: str
    why: str

@function_tool
def create_ticket(issue:str):
  """
  create a ticket in the system for an issue to be resolved
  """
  print(f"Creating ticket for {issue}")
  return "Ticket created ID: 12345"


manager_agent = Agent(
    name="Manager",
    handoff_description="Handles esalated issued that require managerial attention",
    instructions="You handle escalated customer issue that the inital customer service agent could not resolve you will recieve the issue and the reason for escalation. If the issue cannot be immediately resolved for the customer, create a ticket in the system and inform the customer",
    tools=[create_ticket],
    model=model


)

def on_manager_handoff(stx: RunContextWrapper[None], input:ManageEscalation):
  print("Escalating to manager agent:", input.issue)
  print("Reason for escalation:", input.why)

customer_service_agent = Agent(
    name="Customer  Service",
    instructions="You assist customers with general inquiries and the basic troubleshooting."+
    "if the issue cannot be resolved , escalate it to the manager along with the reason why you cannot fix the issue yourself",
    handoffs=[handoff(agent=manager_agent,
                      input_type=ManageEscalation,
                      on_handoff=on_manager_handoff
     )],
    model=model

)
result = await Runner.run(customer_service_agent, "I want a refund but your system won't let process it. the website is just blank for me.")
print(result.final_output)

Escalating to manager agent: refund request and website is blank
Reason for escalation: the system won't let the user process the refund and the website is blank
Creating ticket for refund request and website is blank
OK. I have created a ticket for you with ID 12345. We will investigate the issue and process your refund as soon as possible.

